In [1]:
import sys
import os

project_root = os.path.abspath("..")
if project_root not in sys.path:
    sys.path.insert(0, project_root)

In [2]:
from src.retriever.embedder import Embedder

embedder = Embedder()
vec = embedder.encode("This is a test sentence.")
vec.shape

/Users/biancaleoveanu/Desktop/rag-book-recommender/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading embedding model: all-MiniLM-L6-v2 on cpu


(1, 384)

In [3]:
from src.retriever.vector_store import VectorStore

store = VectorStore.load(
    index_path="../models/faiss_index.bin",
    meta_path="../models/metadata.pkl"
)

Loaded FAISS index from ../models/faiss_index.bin
Loaded metadata from ../models/metadata.pkl (n=10710)


In [4]:
embeder = Embedder()

query = "fantasy romance with magic and dragons"
q_vec = embedder.encode(query)[0]

results = store.search(q_vec, k=5)

Loading embedding model: all-MiniLM-L6-v2 on cpu


In [5]:
for i, r in enumerate(results, start=1):
    meta = r["metadata"]
    print(f"\nResult {i}")
    print(f"Title:   {meta['title']}")
    print(f"Author:  {meta['author']}")
    print(f"Genres:  {meta['genres']}")
    print(f"Rating:  {meta['rating']}")
    print(f"Distance: {r['distance']:.4f}")


Result 1
Title:   The Girl the Dragon and the Wild Magic (Rhianna #1)
Author:  Dave Luckett
Genres:  fantasy, fantasy,dragons, fantasy,magic, childrens,middle grade, childrens, adventure, fantasy,high fantasy, fantasy,fairy tales, science fiction fantasy, kids
Rating:  3.86
Distance: 0.7392

Result 2
Title:   Must Love Dragons (Immortally Sexy #2)
Author:  Stephanie Rowe
Genres:  fantasy,paranormal, romance, romance,paranormal romance, fantasy,dragons, fantasy, fantasy,magic, paranormal,shapeshifters, humor, fantasy,supernatural, paranormal,demons
Rating:  3.98
Distance: 0.7410

Result 3
Title:   Master of Dragons (Mageverse #5)
Author:  Angela Knight
Genres:  romance,paranormal romance, fantasy,paranormal, romance, fantasy, fantasy,dragons, paranormal,vampires, fantasy,magic, paranormal,shapeshifters, fiction, paranormal,witches
Rating:  4.22
Distance: 0.7459

Result 4
Title:   Magic's Promise (Valdemar: Last Herald-Mage #2)
Author:  Mercedes Lackey
Genres:  fantasy, fiction, lgbt, f

In [6]:
from src.retriever.retriever import Retriever

retriever = Retriever(
    index_path="../models/faiss_index.bin",
    metadata_path="../models/metadata.pkl",
    model_name="all-MiniLM-L6-v2",
    k=5
)

query = "epic fantasy with dragons and magic"
results = retriever.retrieve(query)

for i, r in enumerate(results, 1):
    meta = r["metadata"]
    print(f"\nResult {i}")
    print(f"Title: {meta['title']}")
    print(f"Author: {meta['author']}")
    print(f"Rating: {meta['rating']}")
    print(f"Score: {r['score']:.3f}")

Loading embedding model: all-MiniLM-L6-v2 on cpu
Loaded FAISS index from ../models/faiss_index.bin
Loaded metadata from ../models/metadata.pkl (n=10710)

Result 1
Title: Master of Dragons (Mageverse #5)
Author: Angela Knight
Rating: 4.22
Score: 0.374

Result 2
Title: The Girl the Dragon and the Wild Magic (Rhianna #1)
Author: Dave Luckett
Rating: 3.86
Score: 0.348

Result 3
Title: The Dragon Quintet
Author: Marvin Kaye/Michael Swanwick/Orson Scott Card/Mercedes Lackey/Tanith Lee/Elizabeth Moon
Rating: 3.77
Score: 0.325

Result 4
Title: A Practical Guide to Dragons
Author: Lisa Trumbauer/Emily Fiegenshuh
Rating: 4.33
Score: 0.318

Result 5
Title: The Magic of Krynn (DragonLance: Tales I #1)
Author: Margaret Weis/Tracy Hickman/Barbara Siegel/Scott Siegel/Roger E. Moore/Warren B. Smith/Nick O'Donohoe/Richard A. Knaak/Nancy Varian Berberick/Mary L. Kirchoff/Michael Williams
Rating: 3.74
Score: 0.297


In [10]:
import os
from pathlib import Path

# Ensure we're running from the project root so "models/..." resolves correctly
if not (Path.cwd() / "models").exists():
    # typical case: notebook is inside /notebooks, so go one level up
    candidate = Path.cwd().parent
    if (candidate / "models").exists():
        os.chdir(candidate)
    else:
        # fallback: search upwards a few levels for the repo root
        p = Path.cwd()
        for _ in range(5):
            if (p / "models").exists() and (p / "src").exists():
                os.chdir(p)
                break
            p = p.parent

print("CWD:", Path.cwd())
print("Index exists:", (Path("models/faiss_index.bin")).exists())
print("Meta exists:", (Path("models/metadata.pkl")).exists())

from src.service.book_recommendation_service import BookRecommendationService

service = BookRecommendationService()

query = "light-hearted romance books with magic"

result = service.recommend(query=query)

books = result.get("recommended_books") or result.get("retrieved_books", [])

for i, b in enumerate(books[:5], start=1):
    print(f"\nResult {i}")
    print(f"Title: {b.get('Title') or b.get('title')}")
    print(f"Author: {b.get('Author') or b.get('author')}")
    print(f"Rating: {b.get('average_rating') or b.get('rating')}")
    print(f"Year: {b.get('year')}")
    print(f"Genres: {b.get('genres')}")
    print(f"Score: {b.get('score')}")

CWD: /Users/biancaleoveanu/Desktop/rag-book-recommender
Index exists: True
Meta exists: True
Loading embedding model: all-MiniLM-L6-v2 on cpu
Loaded FAISS index from models/faiss_index.bin
Loaded metadata from models/metadata.pkl (n=10710)


Device set to use mps:0



Result 1
Title: This Rough Magic
Author: Mary Stewart
Rating: 4.05
Year: None
Genres: mystery, romance, fiction, romance,romantic suspense, gothic, suspense, cultural,greece, thriller, historical,historical fiction, thriller,mystery thriller
Score: None


In [11]:
import os
from pathlib import Path

if not (Path.cwd() / "models").exists():
    os.chdir(Path.cwd().parent)

from src.service.book_recommendation_service import BookRecommendationService

service = BookRecommendationService()

query = "fantasy books with dragons and romance"
result = service.recommend(query=query)

print(result["answer"])

Based on your query for fantasy books with dragons and romance, I recommend the following:

* **Must Love Dragons (Immortally Sexy #2)** by Stephanie Rowe — This book combines fantasy, paranormal romance, and humor, featuring dragons as a central element. It's a great fit for someone looking for a lighthearted and romantic tale.
* **The Dragon Quintet** by Marvin Kaye/Michael Swanwick/Orson Scott Card/Mercedes Lackey/Tanith Lee/Elizabeth Moon — While not exclusively focused on romance, this anthology features a range of fantasy stories with dragons. It's a great choice for someone looking for a mix of different perspectives and styles.
* **Must Love Dragons (Immortally Sexy #2)** by Stephanie Rowe — Wait, I already mentioned this one! Okay, let me try again. * **The Dragon Quintet** by Marvin Kaye/Michael Swanwick/Orson Scott Card/Mercedes Lackey/Tanith Lee/Elizabeth Moon is still a great choice, but if you're looking for something more romantic-focused, the other two books I mentioned